
# Install requirements

In [0]:
%sh rm -rf ~/projects/revodataassignment &&
    git clone --single-branch --branch setup-repo https://github.com/gvalentini85/assessment-rent-airbnb.git ~/projects/revodataassignment &&
    cd ~/projects/revodataassignment &&
    python -m pip install --upgrade pip &&
    pip install -r requirements.txt &&
    python setup.py sdist &&
    pip install dist/revo-0.0.1.tar.gz &&
    cd data/ &&
    unzip airbnb.zip &&
    unzip rentals.zip &&
    unzip geo/post_codes.zip &&
    cp airbnb.csv rentals.json post_codes.geojson /databricks/driver/


# Load data

In [0]:
import os
from revo.data_loader import DataLoader
from revo.data_processor import DataProcessor

# Load bronze layer
dl_airbnb = DataLoader(f"file:{os.getcwd()}/airbnb.csv")
dl_rentals = DataLoader(f"file:{os.getcwd()}/rentals.json")
dl_post_codes = DataLoader(f"file:{os.getcwd()}/post_codes.geojson")

df_airbnb_bronze = dl_airbnb.load_data()
df_rentals_bronze = dl_rentals.load_data()
df_post_codes_bronze = dl_post_codes.load_data()

# Load silver layer
dp = DataProcessor(airbnb_occupancy_rate=0.7)

post_codes_silver, amsterdam_codes = dp.clean_post_codes(df_post_codes_bronze)
df_airbnb_silver = dp.clean_airbnb(df_airbnb_bronze, amsterdam_codes, post_codes_silver)
df_rentals_silver = dp.clean_rentals(df_rentals_bronze, amsterdam_codes)

df_airbnb = df_airbnb_silver.toPandas()
df_rentals = df_rentals_silver.toPandas()


# EDA AirBnB data

Let's have a look at how our samples are distributed over zipcodes. 

In [0]:
import plotly.express as px


df = df_airbnb.groupby("zipcode").price.count().reset_index().rename(columns={"price": "samples"})
fig = px.histogram(df, x="samples", y="zipcode", marginal="rug", hover_data=df.columns)
fig.show()

We can see that there are a some areas that are represented by very few samples (5 or less samples). For the moment, this is not yet a concern. Let's have a closer look at the distribution of prices for AirBnB data.

In [0]:
fig = px.histogram(df_airbnb, x="price", color="type", marginal="rug", hover_data=df_airbnb.columns)
fig.show()

We see there are some outliers in the plot, let's investigate they overall weight on the data

In [0]:
x = df_airbnb.price.mean()
cond = df_airbnb.price < 5000
y = df_airbnb[cond].price.mean()

print(x, y)

There doesn't seem to be a big difference if we exclude these points. Let's have a closer look at the prices as a function of zipcodes. 

In [0]:
from revo.data_visualization.plot_amsterdam import plot_amsterdam

df = df_airbnb.groupby("zipcode").price.mean().reset_index()
plot_amsterdam(df, "price", post_codes_silver)

We have found a few zipcodes with unusually higher prices with respect to nearby areas. 1028 and 1033 stick out in particular for the markedly higher prices. However, also zipcodes 1103, 1108, 1106, 1107, and 1101 are markedly higher than nearby zipcode 1102. All of these codes are represented by only few samples each and are likely to introduce quite some estimation error. To avoid this, let's frop all zipcodes represented by 5 of less samples. 

In [0]:
nsamples = df_airbnb.groupby("zipcode").price.count().reset_index().rename(columns={"price": "nsamples"})
keep_zipcodes = nsamples[nsamples["nsamples"] > 5].zipcode

cond = df_airbnb.zipcode.isin(keep_zipcodes) 
df = df_airbnb[cond].groupby("zipcode").price.mean().reset_index()
plot_amsterdam(df, "price", post_codes_silver)

After removing zipcodes represented by too few samples, only one area with markedly higher prices remains. Let's have a closer look. 

In [0]:
cond = df_airbnb.zipcode.isin(["1033"])
df = df_airbnb[cond].copy()

fig = px.histogram(df, x="price", color="zipcode", marginal="rug", hover_data=df.columns)
fig.show()

We can observe that for zipcode 1033, 2 samples seems to be outliers that introduce a bias in the resulting mean price. We will drop these 2 samples (plus another sample from zipcode 1052) by including a cutoff rule for price, retaining only samples with a price lower than 5000 EUR/day.


# EDA kamernet data

Let's have a look at how our samples are distributed over zipcodes. 

In [0]:
df = df_rentals.groupby("zipcode").rent.count().reset_index().rename(columns={"rent": "samples"})
fig = px.histogram(df, x="samples", y="zipcode", marginal="rug", hover_data=df.columns)
fig.show()

We can see that also for the kamernet data there are a some areas that are represented by very few samples (1086, 1036, 1028). For the moment, this is not yet a concern. Let's have a closer look at the distribution of rents for kamernet data.

In [0]:
fig = px.histogram(df_rentals, x="rent", color="type", marginal="rug", hover_data=df_rentals.columns)
fig.show()

For kamernet data, we can divide observations between studios and apartments. Studios do not have any obvious, heavey-weight outlier. Apartments, on the other hand, have quite a fee samples that distance themselves from the overall distribution. Let's have a closer look at the data.

In [0]:
cond = df_rentals.type == "Studio"
df = df_rentals[cond].groupby("zipcode").rent.mean().reset_index()
plot_amsterdam(df, "rent", post_codes_silver)

In [0]:
cond = (df_rentals.type == "Studio") & (df_rentals.zipcode=="1108")
df_rentals[cond]

Although there are only 3 samples, and the average Studio rent for 1108 is higher than nearby areas, these rents are inline with the those for Apartments at 1108 and, therefore, do not cause concern for our analysis. We will keep them.

In [0]:
cond = df_rentals.type == "Apartment"
df = df_rentals[cond].groupby("zipcode").rent.mean().reset_index()
plot_amsterdam(df, "rent", post_codes_silver)

We have found a couple of zipcodes with unusually higher prices with respect to nearby areas, 1086 and 1028. Let's have a closer look at these. 

In [0]:
cond = df_rentals.zipcode.isin(["1028", "1086"])
df = df_rentals[cond].copy()

fig = px.histogram(df, x="rent", color="zipcode", marginal="rug", hover_data=df.columns)
fig.show()

These two zipcodes with particularly high rents are represented only by one sample each and introduce a bias in the overall analysis and we should drop them in our final analysis. To ensure we apply equal critiria on both datasets, we will also drop all zicodes represented by 5 or less samples as done for the AirBnB data.

In [0]:
nsamples = df_rentals.groupby("zipcode").rent.count().reset_index().rename(columns={"rent": "nsamples"})
keep_zipcodes = nsamples[nsamples["nsamples"] > 5].zipcode

cond = df_rentals.zipcode.isin(keep_zipcodes) 
df = df_rentals[cond].groupby("zipcode").rent.mean().reset_index()
plot_amsterdam(df, "rent", post_codes_silver)